Plot the SST for the global Ocean.    
Prepared to work with MODIS-Terra SST 4 µm nightime data, at 9 km resolution.

In [1]:
import sys
sys.path.insert(0, '..')
import os
import glob
import numpy as np
from filament import filament
import datetime
import cmocean
import logging
import netCDF4
import calendar
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
from importlib import reload
reload(filament)

<module 'filament.filament' from '../filament/filament.py'>

## Logger

In [2]:
logger = logging.getLogger("SSTanom")
logger.setLevel(logging.INFO)
logging.info("Starting")

## Files and directories

In [3]:
year = 2019
figdir = "/data/SST/Global/figures/"
monthlydir = "/data/SST/Global/monthly/{}".format(year)
climdir = "/data/SST/Global/monthly_clim/"
if os.path.isdir(monthlydir):
    monthfilelist = sorted(glob.glob(os.path.join(monthlydir, "T{0}*.L3m_MO_SST4_sst4_9km.nc").format(year)))
else:
    logger.error("Directory does not exist")
    
climfilelist = sorted(glob.glob(os.path.join(climdir, "T*.L3m_MC_SST4_sst4_9km.nc")))
logger.info("Found {} monthly files".format(len(monthfilelist)))

INFO:SSTanom:Found 10 monthly files


## Create dictionary for the monthly climatology files
Each entry contains the month number (from 1 to 12) and the corresponding climatological file.

In [4]:
climDict = {}
for climfiles in climfilelist:
    logger.debug("Working on file {}".format(climfiles))
    with netCDF4.Dataset(climfiles, "r") as nc:
        time_coverage_start = nc.time_coverage_start[:]
        time_coverage_end = nc.time_coverage_end[:]
    date_start = datetime.datetime.strptime(time_coverage_start, "%Y-%m-%dT%H:%M:%S.000Z")
    date_end = datetime.datetime.strptime(time_coverage_end, "%Y-%m-%dT%H:%M:%S.000Z")
    m = date_end.month
    climDict[m] = climfiles

In [5]:
climDict

{1: '/data/SST/Global/monthly_clim/T20010012019031.L3m_MC_SST4_sst4_9km.nc',
 2: '/data/SST/Global/monthly_clim/T20000322019059.L3m_MC_SST4_sst4_9km.nc',
 3: '/data/SST/Global/monthly_clim/T20000612019090.L3m_MC_SST4_sst4_9km.nc',
 4: '/data/SST/Global/monthly_clim/T20000922019120.L3m_MC_SST4_sst4_9km.nc',
 5: '/data/SST/Global/monthly_clim/T20001222019151.L3m_MC_SST4_sst4_9km.nc',
 6: '/data/SST/Global/monthly_clim/T20001532018181.L3m_MC_SST4_sst4_9km.nc',
 7: '/data/SST/Global/monthly_clim/T20001832019212.L3m_MC_SST4_sst4_9km.nc',
 8: '/data/SST/Global/monthly_clim/T20002142019243.L3m_MC_SST4_sst4_9km.nc',
 9: '/data/SST/Global/monthly_clim/T20002452019273.L3m_MC_SST4_sst4_9km.nc',
 10: '/data/SST/Global/monthly_clim/T20002752019304.L3m_MC_SST4_sst4_9km.nc',
 11: '/data/SST/Global/monthly_clim/T20003062018334.L3m_MC_SST4_sst4_9km.nc',
 12: '/data/SST/Global/monthly_clim/T20003362018365.L3m_MC_SST4_sst4_9km.nc'}

## Loop on the monthly files
The goal is to find the correct monthly climatology files, knowing that 
* the first and last years are not always the same, 
* the days of the years may also change.

## Plot preparation
### Projection

In [12]:
SSTmonth = filament.SST()
SSTmonth.read_from_oceancolorL3(climDict[1])

In [13]:
m = Basemap(projection='ortho',lon_0=-52.,lat_0=10,resolution='c')
llon, llat = np.meshgrid(SSTmonth.lon, SSTmonth.lat)
lonp, latp = m(llon, llat)
lonp[lonp==lonp.max()] = np.nan
latp[latp==latp.max()] = np.nan

### Plotting function

In [61]:
def make_monthly_subplot(lon, lat, SSTanom, month, NN=1, fgcolor="k"):
    plt.title(calendar.month_name[month], fontsize=18, color=fgcolor)
    m.drawcoastlines(linewidth=.2)
    pcm = m.pcolormesh(lon[::NN], lat[::NN], SSTanom[::NN],
                 zorder=3, cmap=plt.cm.RdBu_r, vmin=-2.5, vmax=2.5)
    # m.fillcontinents(color='grey')
    #plt.colorbar(extend="both", shrink=.7)
    m.warpimage("world.topo.bathy.200403.3x5400x2700.jpg", zorder=2)
    #mpol2.drawcoastlines()
    #plt.savefig(os.path.join(figdir, "SST_anom_test"), dpi=300, bbox_inches="tight")
    # plt.show()
    #plt.close()
    return pcm

In [63]:
plt.close("all")
bgcolor = "black"
fgcolor = "white"
fig = plt.figure(figsize=(13, 10), facecolor=bgcolor)

for imonth, monthlyfile in enumerate(monthfilelist):
    logger.debug("Working on file {}".format(monthlyfile))
    
    with netCDF4.Dataset(monthlyfile, "r") as nc:
        time_coverage_start = nc.time_coverage_start[:]
        time_coverage_end = nc.time_coverage_end[:]
    date_start = datetime.datetime.strptime(time_coverage_start, "%Y-%m-%dT%H:%M:%S.000Z")
    date_end = datetime.datetime.strptime(time_coverage_end, "%Y-%m-%dT%H:%M:%S.000Z")
    date_mid = date_start + 0.5 * (date_end - date_start)
    month = date_mid.month
    logger.info("Working on month {}/12".format(month))
    
    # Read data (lon, lat and SST)
    SSTmonth = filament.SST()
    SSTmonth.read_from_oceancolorL3(monthlyfile)
    SSTclim = filament.SST()
    SSTclim.read_from_oceancolorL3(climDict[month])
    
    # Compute anomalies
    SSTanom = SSTmonth.field - SSTclim.field
    
    ax = plt.subplot(3, 4, imonth+1)
    pcm = make_monthly_subplot(lonp, latp, SSTanom, imonth+1, NN=10, fgcolor=fgcolor)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.9, 0.15, 0.03, 0.7])
normanom = mpl.colors.Normalize(vmin=-2.5, vmax=2.5)
cb1 = mpl.colorbar.ColorbarBase(cbar_ax, cmap=plt.cm.RdBu_r,
                                norm=normanom, orientation='vertical', 
                                extend="both")

cb1.set_label("$^{\circ}$C", rotation=0, ha="left", fontsize=18, 
              color=fgcolor, fontweight="bold")

cb1.ax.tick_params(axis='y', colors=fgcolor)


cb1_ytl = cb1.ax.get_yticklabels()
cb1.ax.set_yticklabels(cb1_ytl, color=fgcolor, 
                       fontweight="bold", fontsize=16)


fig.suptitle('Sea surface temperature anomalies $-$ {}'.format(year), 
             fontsize=24, color=fgcolor) 
plt.savefig(os.path.join(figdir, "SSTanomalies{}_01".format(year)), 
            facecolor=fig.get_facecolor(),
            dpi=300, bbox_inches="tight")
#plt.show()
plt.close()

INFO:SSTanom:Working on month 1/12
INFO:SSTanom:Working on month 2/12
INFO:SSTanom:Working on month 3/12
INFO:SSTanom:Working on month 4/12
INFO:SSTanom:Working on month 5/12
INFO:SSTanom:Working on month 6/12
INFO:SSTanom:Working on month 7/12
INFO:SSTanom:Working on month 8/12
INFO:SSTanom:Working on month 9/12
INFO:SSTanom:Working on month 10/12
